In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

# FMNIST 데이터셋 로드
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

# 픽셀 값 정규화
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

# 모델 구조 정의
model = tf.keras.Sequential([
  layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
  layers.MaxPooling2D((2, 2)),
  layers.Flatten(),
  layers.Dense(64, activation='relu'),
  layers.Dense(10)
])

# 모델 컴파일
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# 모델 학습
history = model.fit(x_train.reshape(-1, 28, 28, 1), y_train, 
                    epochs=10, 
                    validation_data=(x_test.reshape(-1, 28, 28, 1), y_test))

# 학습/검증 손실 출력
print("Train loss:", history.history['loss'][-1])
print("Validation loss:", history.history['val_loss'][-1])

In [ ]:
# 입력 레이어 정의
inputs = tf.keras.Input(shape=(28, 28, 1))

# 컨볼루션 레이어 정의
x = layers.Conv2D(32, (3, 3), activation='relu')(inputs)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)

# 완전 연결 레이어 정의
x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(10)(x)

# 모델 정의
model = tf.keras.Model(inputs=inputs, outputs=outputs)

# 모델 컴파일
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# 모델 학습
history = model.fit(x_train.reshape(-1, 28, 28, 1), y_train, 
                    epochs=10, 
                    validation_data=(x_test.reshape(-1, 28, 28, 1), y_test))

# 학습/검증
print("Train loss:", history.history['loss'][-1])
print("Validation loss:", history.history['val_loss'][-1])

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam

# CIFAR10 데이터셋 로드 및 전처리
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Sequential API를 사용한 모델
model_seq = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

model_seq.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

history_seq = model_seq.fit(x_train, y_train, epochs=10, batch_size=128, validation_data=(x_test, y_test))

# Functional API를 사용한 모델
inputs = Input(shape=(32,32,3))
x = Conv2D(32, (3,3), activation='relu')(inputs)
x = MaxPooling2D((2,2))(x)
x = Conv2D(64, (3,3), activation='relu')(x)
x = MaxPooling2D((2,2))(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(10, activation='softmax')(x)

model_func = Model(inputs=inputs, outputs=outputs)

model_func.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

history_func = model_func.fit(x_train, y_train, epochs=10, batch_size=128, validation_data=(x_test, y_test))

# 학습/검증 손실 출력
print("Sequential API 모델")
print("Train loss:", history_seq.history['loss'][-1])
print("Validation loss:", history_seq.history['val_loss'][-1])

print("\nFunctional API 모델")
print("Train loss:", history_func.history['loss'][-1])
print("Validation loss:", history_func.history['val_loss'][-1]))

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# CIFAR10 데이터셋 로드 및 전처리
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# 사전 학습된 ResNet50 모델 로드
resnet = ResNet50(include_top=False, weights='imagenet', input_shape=(32,32,3))

# ResNet50 모델의 일부 층을 동결
for layer in resnet.layers[:100]:
    layer.trainable = False

# 전이 학습 모델 정의
model = Sequential([
    resnet,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(10, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# 조기 종료를 위한 콜백 함수
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# 모델 학습
history = model.fit(x_train, y_train, epochs=20, batch_size=128, validation_data=(x_test, y_test), callbacks=[early_stopping])

# 학습/검증 손실 출력
print("Train loss:", history.history['loss'][-1])
print("Validation loss:", history.history['val_loss'][-1])